In [44]:
# =============================================================================
# Spatial Econometrics Analysis for Russian Regions (Linux/Codespaces Version)
# =============================================================================

# 1. ПОДКЛЮЧЕНИЕ БИБЛИОТЕК
# Мы используем suppressPackageStartupMessages, чтобы консоль была чище
suppressPackageStartupMessages({
  library(readxl)      # Чтение Excel
  library(dplyr)       # Обработка данных
  library(sf)          # Работа с картами (Simple Features)
  library(spdep)       # Пространственная статистика
  library(spatialreg)  # Регрессии
  library(ggplot2)     # Графики
  # library(rgeoda)    # (Опционально, если удалось установить)
})

# =============================================================================
# 1. DATA LOADING AND PREPARATION
# =============================================================================

cat("=== 1. Loading and Preparing Data ===\n\n")

# В Linux/Codespaces рабочей папкой автоматически считается корень проекта.
# setwd(...) - УДАЛЕНО, чтобы не вызывать ошибку.

# Проверяем, на месте ли файл
excel_file <- "Umempl_spatial.xlsx"
if (!file.exists(excel_file)) {
  stop("ОШИБКА: Файл 'Umempl_spatial.xlsx' не найден в папке проекта!\nЗагрузи его в ту же папку, где лежит скрипт.")
}

# Загружаем данные
data <- read_excel(excel_file)

cat("Data dimensions:", nrow(data), "rows x", ncol(data), "columns\n")
cat("Years:", unique(data$Year), "\n")
cat("Number of regions:", length(unique(data$Region)), "\n\n")

# Оставляем только 2019 год
data_2019 <- data %>% filter(Year == 2019)
cat("2019 data: ", nrow(data_2019), "regions\n\n")


# =============================================================================
# 2. GET RUSSIAN REGIONS SHAPEFILE (Natural Earth)
# =============================================================================

cat("\n=== 2. Loading Russian Regions Shapefile (Natural Earth) ===\n\n")

# Параметры скачивания карты
ne_url <- "https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_1_states_provinces.zip"
ne_zip <- "ne_10m_admin_1_states_provinces.zip"
ne_dir <- "ne_10m_admin_1_states_provinces"

# Увеличиваем таймаут до 300 секунд (интернет в облаке может "икать")
options(timeout = 300)

# Скачиваем архив, если его нет
if (!file.exists(ne_zip)) {
  cat("Downloading Natural Earth shapefile (please wait)...\n")
  # В Linux используем mode = "wb" обязательно
  tryCatch({
    download.file(ne_url, ne_zip, mode = "wb", quiet = TRUE)
  }, error = function(e) {
    stop("Не удалось скачать карту. Проверь интернет или загрузи файл вручную.")
  })
}

# Распаковываем
if (!dir.exists(ne_dir)) {
  cat("Extracting shapefile...\n")
  unzip(ne_zip, exdir = ne_dir)
}

# Читаем карту
shp_path <- file.path(ne_dir, "ne_10m_admin_1_states_provinces.shp")
ne_shp <- st_read(shp_path, quiet = TRUE)

# Фильтруем только Россию
russia <- ne_shp %>% filter(admin == "Russia")

# --- ВАЖНО: ИСПРАВЛЕНИЕ ПРОЕКЦИИ ---
# Используем ту же проекцию LAEA (Lambert), чтобы Чукотка не улетала
russia_crs <- "+proj=laea +lat_0=60 +lon_0=100 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
russia <- st_transform(russia, crs = russia_crs)

# Лечим геометрию (в Linux это критично для sf)
russia <- st_make_valid(russia)

cat("Shapefile contains", nrow(russia), "Russian administrative units\n")


# =============================================================================
# 3. MERGE DATA WITH SHAPEFILE
# =============================================================================

cat("\n=== 3. Merging Data ===\n\n")

name_mapping <- c(
  # --- Север и Архангельск (Они у тебя разбиты в файле!) ---
  'Arkhangelsk Oblast (excluding the Nenets Autonomous Okrug)' = "Arkhangel'sk",
  'Nenets Autonomous Okrug (Arkhangelsk Oblast)' = 'Nenets',

  # --- Тюмень (Только юг) ---
  'Tyumen Oblast' = "Tyumen'",

  # --- Республики и сложные названия ---
  'Republic of Adygea (Adygea)' = 'Adygey',
  'Republic of Bashkortostan' = 'Bashkortostan',
  'Republic of Buryatia' = 'Buryat',
  'Republic of Altai' = 'Gorno-Altay',
  'Republic of Dagestan' = 'Dagestan',
  'Republic of Ingushetia' = 'Ingush',
  'Kabardino-Balkar Republic' = 'Kabardin-Balkar',
  'Republic of Kalmykia' = 'Kalmyk',
  'Karachay-Cherkess Republic' = 'Karachay-Cherkess',
  'Republic of Karelia' = 'Karelia',
  'Republic of Komi' = 'Komi',
  'Republic of Mari El' = 'Mariy-El',
  'Republic of Mordovia' = 'Mordovia',
  'Republic of Sakha (Yakutia)' = 'Sakha (Yakutia)',
  'Republic of North Ossetia–Alania' = 'North Ossetia',
  'Republic of Tatarstan (Tatarstan)' = 'Tatarstan',
  'Republic of Tyva' = 'Tuva',
  'Udmurt Republic' = 'Udmurt',
  'Republic of Khakassia' = 'Khakass',
  'Chechen Republic' = 'Chechnya',
  'Chuvash Republic - Chuvashia' = 'Chuvash',
  'Republic of Crimea' = 'Crimea',

  # --- Края и области ---
  'Altai Krai' = 'Altay',
  'Krasnodar Krai' = 'Krasnodar',
  'Krasnoyarsk Krai' = 'Krasnoyarsk',
  'Primorsky Krai' = "Primor'ye",
  'Stavropol Krai' = "Stavropol'",
  'Khabarovsk Krai' = 'Khabarovsk',
  'Zabaykalsky Krai' = 'Chita',
  'Kamchatka Krai' = 'Kamchatka',
  'Perm Krai' = "Perm'",
  'Amur Oblast' = 'Amur',
  'Astrakhan Oblast' = "Astrakhan'",
  'Belgorod Oblast' = 'Belgorod',
  'Bryansk Oblast' = 'Bryansk',
  'Vladimir Oblast' = 'Vladimir',
  'Volgograd Oblast' = 'Volgograd',
  'Vologda Oblast' = 'Vologda',
  'Voronezh Oblast' = 'Voronezh',
  'Ivanovo Oblast' = 'Ivanovo',
  'Irkutsk Oblast' = 'Irkutsk',
  'Kaliningrad Oblast' = 'Kaliningrad',
  'Kaluga Oblast' = 'Kaluga',
  'Kemerovo Oblast - Kuzbass' = 'Kemerovo',
  'Kirov Oblast' = 'Kirov',
  'Kostroma Oblast' = 'Kostroma',
  'Kurgan Oblast' = 'Kurgan',
  'Kursk Oblast' = 'Kursk',
  'Leningrad Oblast' = 'Leningrad',
  'Lipetsk Oblast' = 'Lipetsk',
  'Magadan Oblast' = 'Maga Buryatdan',
  'Moscow Oblast' = 'Moskovskaya',
  'Murmansk Oblast' = 'Murmansk',
  'Nizhny Novgorod Oblast' = 'Nizhegorod',
  'Novgorod Oblast' = 'Novgorod',
  'Novosibirsk Oblast' = 'Novosibirsk',
  'Omsk Oblast' = 'Omsk',
  'Orenburg Oblast' = 'Orenburg',
  'Oryol Oblast' = 'Orel',
  'Penza Oblast' = 'Penza',
  'Pskov Oblast' = 'Pskov',
  'Rostov Oblast' = 'Rostov',
  'Ryazan Oblast' = "Ryazan'",
  'Samara Oblast' = 'Samara',
  'Saratov Oblast' = 'Saratov',
  'Sakhalin Oblast' = 'Sakhalin',
  'Sverdlovsk Oblast' = 'Sverdlovsk',
  'Smolensk Oblast' = 'Smolensk',
  'Tambov Oblast' = 'Tambov',
  'Tver Oblast' = "Tver'",
  'Tomsk Oblast' = 'Tomsk',
  'Tula Oblast' = 'Tula',
  'Ulyanovsk Oblast' = "Ul'yanovsk",
  'Chelyabinsk Oblast' = 'Chelyabinsk',
  'Yaroslavl Oblast' = "Yaroslavl'",
  'Jewish Autonomous Oblast' = 'Yevrey',
  'Chukotka Autonomous Okrug' = 'Chukchi Autonomous Okrug',

  # --- Города ---
  'Moscow' = 'Moskva',
  'Saint Petersburg' = 'City of St. Petersburg',
  'City of federal significance Sevastopol' = 'Sevastopol'
)

# Create mapped name column in data
data_2019$ne_name <- ifelse(
  data_2019$geoname_name %in% names(name_mapping),
  name_mapping[data_2019$geoname_name],
  data_2019$geoname_name
)

data_2019

cat("Matching regions using name mapping...\n")

# Join using mapped names
spatial_data_2 <- russia %>%
  left_join(data_2019, by = c("name" = "ne_name"))

# Check how many matched (via the y variable)
matched_count <- sum(!is.na(spatial_data_2$y))
cat("Matched regions:", matched_count, "out of", nrow(data_2019), "\n")

# Keep ALL matched regions (no filtering)
spatial_data_clean <- spatial_data_2 %>% filter(!is.na(y))
spatial_data_clean <- st_make_valid(spatial_data_clean)
cat("Regions with complete spatial data:", nrow(spatial_data_clean), "\n")



=== 1. Loading and Preparing Data ===

Data dimensions: 276 rows x 7 columns
Years: 2018 2019 2020 
Number of regions: 92 

2019 data:  92 regions


=== 2. Loading Russian Regions Shapefile (Natural Earth) ===



Shapefile contains 86 Russian administrative units

=== 3. Merging Data ===



Year,Region,y,x1,x2,x3,geoname_name,ne_name
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
2019,Центральный федеральный округ,2.9,424.1,6.4,40336196,Central Federal District,Central Federal District
2019,Белгородская область,3.9,415.1,6.7,1551920,Belgorod Oblast,Belgorod
2019,Брянская область,3.8,567.9,6.5,1193180,Bryansk Oblast,Bryansk
2019,Владимирская область,4.0,555.8,5.9,1376582,Vladimir Oblast,Vladimir
2019,Воронежская область,3.5,470.8,6.9,2337457,Voronezh Oblast,Voronezh
2019,Ивановская область,3.7,590.4,5.9,959459,Ivanovo Oblast,Ivanovo
2019,Калужская область,3.7,526.0,6.8,1052781,Kaluga Oblast,Kaluga
2019,Костромская область,4.0,554.1,6.0,601815,Kostroma Oblast,Kostroma
2019,Курская область,3.9,537.2,6.5,1101403,Kursk Oblast,Kursk


Matching regions using name mapping...
Matched regions: 83 out of 92 
Regions with complete spatial data: 83 


In [35]:

# =============================================================================
# 4. CREATE SPATIAL WEIGHTS MATRIX - USING KNN (KEY DIFFERENCE!)
# =============================================================================

cat("\n=== 4. Creating Spatial Weights Matrix (KNN) ===\n\n")

# Get centroids of each region
centroids <- st_centroid(spatial_data_clean)
coords <- st_coordinates(centroids)

cat("Coordinate range:\n")
cat("  Longitude:", round(min(coords[,1]), 2), "to", round(max(coords[,1]), 2), "\n")
cat("  Latitude:", round(min(coords[,2]), 2), "to", round(max(coords[,2]), 2), "\n")



K <- 5
cat("\nUsing K-Nearest Neighbors with K =", K, "\n")

# Create KNN neighbors
knn_nb <- knn2nb(knearneigh(coords, k = K))

cat("\nKNN Neighbor summary:\n")
cat("  - Total regions:", length(knn_nb), "\n")
cat("  - Each region has exactly", K, "neighbors\n")
cat("  - No isolated regions!\n")

# Compare with Queen contiguity
nb_queen <- poly2nb(spatial_data_clean, queen = TRUE)
isolated_queen <- sum(card(nb_queen) == 0)
cat("\nComparison with Queen contiguity:\n")
cat("  - Queen would have", isolated_queen, "isolated regions\n")
cat("  - Queen neighbor counts:", min(card(nb_queen)), "to", max(card(nb_queen)), "\n")

# Create row-standardized weights from KNN
W <- nb2listw(knn_nb, style = "W")
cat("\nSpatial weights created (row-standardized, KNN-based)\n")



=== 4. Creating Spatial Weights Matrix (KNN) ===



Warning message in st_centroid.sf(spatial_data_clean):
“st_centroid assumes attributes are constant over geometries of x”


Coordinate range:
  Longitude: -4455982 to 3046378 
  Latitude: -966969.6 to 2372315 

Using K-Nearest Neighbors with K = 5 

KNN Neighbor summary:
  - Total regions: 83 
  - Each region has exactly 5 neighbors
  - No isolated regions!


Warning message in poly2nb(spatial_data_clean, queen = TRUE):
“some observations have no neighbours;
if this seems unexpected, try increasing the snap argument.”
Warning message in poly2nb(spatial_data_clean, queen = TRUE):
“neighbour object has 4 sub-graphs;
if this sub-graph count seems unexpected, try increasing the snap argument.”



Comparison with Queen contiguity:
  - Queen would have 2 isolated regions
  - Queen neighbor counts: 0 to 9 

Spatial weights created (row-standardized, KNN-based)


In [36]:
# =============================================================================
# 5. MORAN'S I TEST
# =============================================================================

cat("\n=== 5. Moran's I Test ===\n\n")

moran_test <- moran.test(spatial_data_clean$y, W)
cat("Moran's I Test for Unemployment Rate:\n")
print(moran_test)

cat("\nInterpretation:\n")
if (moran_test$p.value < 0.05) {
  if (moran_test$estimate["Moran I statistic"] > 0) {
    cat("Significant POSITIVE spatial autocorrelation detected.\n")
    cat("Moran's I =", round(moran_test$estimate["Moran I statistic"], 4), "\n")
  }
}


=== 5. Moran's I Test ===

Moran's I Test for Unemployment Rate:

	Moran I test under randomisation

data:  spatial_data_clean$y  
weights: W    

Moran I statistic standard deviate = 9.1934, p-value < 2.2e-16
alternative hypothesis: greater
sample estimates:
Moran I statistic       Expectation          Variance 
      0.490865749      -0.012195122       0.002994283 


Interpretation:
Significant POSITIVE spatial autocorrelation detected.
Moran's I = 0.4909 


In [37]:
# =============================================================================
# 7. SPATIAL REGRESSION MODELS
# =============================================================================

cat("\n=== 7. Spatial Regression Models ===\n\n")

# Prepare variables
spatial_data_clean$log_x1 <- log(spatial_data_clean$x1 + 1)
spatial_data_clean$log_x2 <- log(spatial_data_clean$x2 + 1)
spatial_data_clean$log_x3 <- log(spatial_data_clean$x3 + 1)

formula_ols <- y ~ log_x1 + log_x2 + log_x3

# OLS
ols_model <- lm(formula_ols, data = spatial_data_clean)
cat("OLS R-squared:", round(summary(ols_model)$r.squared, 4), "\n")
cat("OLS AIC:", round(AIC(ols_model), 2), "\n")

# SAR
sar_model <- lagsarlm(formula_ols, data = spatial_data_clean, listw = W)
cat("\nSAR Rho:", round(coef(sar_model)["rho"], 4), "\n")
cat("SAR AIC:", round(AIC(sar_model), 2), "\n")

# SEM
sem_model <- errorsarlm(formula_ols, data = spatial_data_clean, listw = W)
cat("\nSEM Lambda:", round(coef(sem_model)["lambda"], 4), "\n")
cat("SEM AIC:", round(AIC(sem_model), 2), "\n")



=== 7. Spatial Regression Models ===

OLS R-squared: 0.5205 
OLS AIC: 390.83 



SAR Rho: 0.4082 
SAR AIC: 380.04 

SEM Lambda: 0.5571 
SEM AIC: 375.51 


In [38]:
# =============================================================================
# 8. SAC MODEL (Combined Spatial Lag and Error)
# =============================================================================

cat("\n=== 8. SAC Model (Spatial Lag + Error) ===\n\n")

# SAC Model: y = rho*W*y + X*beta + u, where u = lambda*W*u + epsilon
# This is the most general specification combining both spatial lag and spatial error
sac_model <- sacsarlm(formula_ols, data = spatial_data_clean, listw = W)
cat("SAC Model Summary:\n")
print(summary(sac_model))

cat("\nRho (spatial lag):", round(coef(sac_model)["rho"], 4), "\n")
cat("Lambda (spatial error):", round(coef(sac_model)["lambda"], 4), "\n")
cat("SAC AIC:", round(AIC(sac_model), 2), "\n")



=== 8. SAC Model (Spatial Lag + Error) ===

SAC Model Summary:

Call:sacsarlm(formula = formula_ols, data = spatial_data_clean, listw = W)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.50723 -0.94193 -0.16441  0.76546 11.01971 

Type: sac 
Coefficients: (asymptotic standard errors) 
            Estimate Std. Error z value  Pr(>|z|)
(Intercept) 67.75175    9.31199  7.2758 3.444e-13
log_x1      -4.92389    1.19146 -4.1327 3.586e-05
log_x2      -5.48324    2.19505 -2.4980   0.01249
log_x3      -1.26114    0.27002 -4.6705 3.005e-06

Rho: -0.49042
Asymptotic standard error: 0.25415
    z-value: -1.9296, p-value: 0.053654
Lambda: 0.79644
Asymptotic standard error: 0.084485
    z-value: 9.427, p-value: < 2.22e-16

LR test value: 19.441, p-value: 6.005e-05

Log likelihood: -180.6942 for sac model
ML residual variance (sigma squared): 3.782, (sigma: 1.9447)
Number of observations: 83 
Number of parameters estimated: 7 
AIC: 375.39, (AIC for lm: 390.83)


Rho (spatial lag): -0.49

In [39]:
# =============================================================================
# 9. SDM MODEL (Spatial Durbin Model)
# =============================================================================

cat("\n=== 9. SDM Model (Spatial Durbin Model) ===\n\n")

# SDM Model: y = rho*W*y + X*beta + W*X*theta + epsilon
# Includes spatially lagged explanatory variables (spillover effects from neighbors' X)
sdm_model <- lagsarlm(formula_ols, data = spatial_data_clean, listw = W,
                      type = "mixed")
cat("SDM Model Summary:\n")
print(summary(sdm_model))

cat("\nRho (spatial lag):", round(coef(sdm_model)["rho"], 4), "\n")
cat("SDM AIC:", round(AIC(sdm_model), 2), "\n")



=== 9. SDM Model (Spatial Durbin Model) ===



SDM Model Summary:

Call:lagsarlm(formula = formula_ols, data = spatial_data_clean, listw = W, 
    type = "mixed")

Residuals:
     Min       1Q   Median       3Q      Max 
-4.16129 -1.22235 -0.17913  0.81499 10.84979 

Type: mixed 
Coefficients: (asymptotic standard errors) 
            Estimate Std. Error z value  Pr(>|z|)
(Intercept) 31.80168   13.63237  2.3328   0.01966
log_x1      -5.58242    1.27246 -4.3871 1.149e-05
log_x2      -4.68534    2.40175 -1.9508   0.05108
log_x3      -1.30932    0.28457 -4.6011 4.203e-06
lag.log_x1  -0.16300    2.18989 -0.0744   0.94067
lag.log_x2  10.53771    4.97110  2.1198   0.03402
lag.log_x3   0.96067    0.57620  1.6672   0.09547

Rho: 0.51663, LR test value: 13.785, p-value: 0.00020494
Asymptotic standard error: 0.11955
    z-value: 4.3214, p-value: 1.5507e-05
Wald statistic: 18.674, p-value: 1.5507e-05

Log likelihood: -180.0967 for mixed model
ML residual variance (sigma squared): 4.2722, (sigma: 2.0669)
Number of observations: 83 
Number of p

In [40]:
# =============================================================================
# 10. MODEL COMPARISON
# =============================================================================

cat("\n=== 10. Model Comparison ===\n\n")

# Model comparison
cat("--- Model Comparison ---\n")
model_comparison <- data.frame(
  Model = c("OLS", "SAR", "SEM", "SAC", "SDM"),
  AIC = c(AIC(ols_model), AIC(sar_model), AIC(sem_model), AIC(sac_model), AIC(sdm_model)),
  LogLik = c(as.numeric(logLik(ols_model)),
             as.numeric(logLik(sar_model)),
             as.numeric(logLik(sem_model)),
             as.numeric(logLik(sac_model)),
             as.numeric(logLik(sdm_model)))
)
cat("Model Comparison (lower AIC is better):\n")
print(model_comparison[order(model_comparison$AIC), ])

# Best model
best_model <- model_comparison$Model[which.min(model_comparison$AIC)]
cat("\nBest model based on AIC:", best_model, "\n")




=== 10. Model Comparison ===

--- Model Comparison ---
Model Comparison (lower AIC is better):
  Model      AIC    LogLik
4   SAC 375.3885 -180.6942
3   SEM 375.5070 -181.7535
5   SDM 378.1933 -180.0967
2   SAR 380.0373 -184.0187
1   OLS 390.8291 -190.4146

Best model based on AIC: SAC 


In [41]:

# =============================================================================
# 11. SPATIAL IMPACTS (Direct and Indirect Effects)
# =============================================================================

cat("\n=== 11.Spatial Impacts (SAR Model) ===\n\n")

# For SAR model, calculate direct, indirect, and total effects
# Direct: effect of own X on own Y
# Indirect: spillover effect from neighbors' X on own Y (spatial multiplier)
# Total: Direct + Indirect
impacts_sar <- impacts(sar_model, listw = W, R = 500)
cat("Direct, Indirect, and Total Effects (SAR Model):\n")
print(summary(impacts_sar, zstats = TRUE))




=== 11.Spatial Impacts (SAR Model) ===

Direct, Indirect, and Total Effects (SAR Model):
Impact measures (lag, exact):
                Direct   Indirect     Total
log_x1 dy/dx -4.553746 -2.8998222 -7.453568
log_x2 dy/dx -3.749612 -2.3877508 -6.137363
log_x3 dy/dx -1.402071 -0.8928381 -2.294909
Simulation results ( variance matrix):
Direct:

Iterations = 1:500
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 500 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

               Mean     SD Naive SE Time-series SE
log_x1 dy/dx -4.467 1.1259  0.05035       0.050351
log_x2 dy/dx -3.897 2.2820  0.10205       0.102052
log_x3 dy/dx -1.391 0.2719  0.01216       0.007464

2. Quantiles for each variable:

               2.5%    25%    50%    75%   97.5%
log_x1 dy/dx -6.460 -5.214 -4.475 -3.717 -2.1970
log_x2 dy/dx -8.355 -5.482 -4.012 -2.368  0.4413
log_x3 dy/dx -1.910 -1.571 -1.391 -1.200 -0.8941

Indirect:

Iterations = 1:500


In [47]:
# =============================================================================
# 12. VISUALIZATION (FINAL FIXED VERSION)
# =============================================================================

cat("\n=== 12. Creating Maps and Plots ===\n\n")

# --- ШАГ 1: Подготовка данных для графиков ---
cat("Preparing data for plots...\n")

# 1. Создаем стандартизированные переменные (Именно их не хватало!)
spatial_data_clean$y_std <- as.numeric(scale(spatial_data_clean$y))
spatial_data_clean$lag_y <- lag.listw(W, spatial_data_clean$y)
spatial_data_clean$lag_y_std <- as.numeric(scale(spatial_data_clean$lag_y))

# 2. Считаем кластеры (LISA)
local_m <- localmoran(spatial_data_clean$y, W)

# Центрируем для определения квадрантов
m_y <- spatial_data_clean$y - mean(spatial_data_clean$y)
m_lag_y <- spatial_data_clean$lag_y - mean(spatial_data_clean$lag_y)

# Определяем значимость и типы кластеров
signif <- 0.05
spatial_data_clean$cluster <- "Not Significant"
is_sig <- local_m[, 5] < signif

spatial_data_clean$cluster[is_sig & m_y > 0 & m_lag_y > 0] <- "High-High"
spatial_data_clean$cluster[is_sig & m_y < 0 & m_lag_y < 0] <- "Low-Low"
spatial_data_clean$cluster[is_sig & m_y > 0 & m_lag_y < 0] <- "High-Low"
spatial_data_clean$cluster[is_sig & m_y < 0 & m_lag_y > 0] <- "Low-High"

spatial_data_clean$cluster <- factor(spatial_data_clean$cluster, 
                                     levels = c("High-High", "Low-Low", "High-Low", "Low-High", "Not Significant"))

cat("Clusters and standardized variables calculated.\n")


# --- ШАГ 2: Карта (Безработица) ---
p1 <- ggplot(spatial_data_clean) +
  geom_sf(aes(fill = y), color = "white", size = 0.1) +
  scale_fill_viridis_c(option = "plasma", direction = -1, name = "Unemployment\nRate (%)") +
  labs(title = "Unemployment Rate by Region (2019)") +
  theme_void() +
  theme(legend.position = "right")

# Сохраняем карту
ggsave("Map_Unemployment.png", plot = p1, width = 10, height = 6, dpi = 300, bg = "white")
cat("Map saved as 'Map_Unemployment.png'\n")


# --- ШАГ 3: Moran Scatterplot (График Морана) ---
# Цвета для кластеров
moran_colors <- c(
  "High-High" = "#d7191c", 
  "Low-Low" = "#2c7bb6",   
  "High-Low" = "#fdae61",  
  "Low-High" = "#abd9e9",  
  "Not Significant" = "grey80"
)

p3 <- ggplot(spatial_data_clean, aes(x = y_std, y = lag_y_std)) +
  geom_hline(yintercept = 0, color = "grey50", linetype = "dashed") +
  geom_vline(xintercept = 0, color = "grey50", linetype = "dashed") +
  # Используем linewidth вместо size (чтобы убрать Warning)
  geom_smooth(method = "lm", se = FALSE, color = "black", linewidth = 0.5) +
  geom_point(aes(fill = cluster), shape = 21, color = "black", size = 3, alpha = 0.9) +
  scale_fill_manual(values = moran_colors, name = "LISA Cluster") +
  annotate("text", x = 3, y = 2, label = "HH", size = 5, fontface = "bold", color = "#d7191c") +
  annotate("text", x = -1.5, y = -1.5, label = "LL", size = 5, fontface = "bold", color = "#2c7bb6") +
  labs(
    title = "Moran's I Scatterplot",
    subtitle = sprintf("Moran's I = %.3f", moran_test$estimate["Moran I statistic"]),
    x = "Unemployment (Std)",
    y = "Spatial Lag (Std)"
  ) +
  theme_minimal()

# Сохраняем график
ggsave("Moran_Scatterplot.png", plot = p3, width = 8, height = 6, dpi = 300, bg = "white")
cat("Scatterplot saved as 'Moran_Scatterplot.png'\n")
cat("Done! Check the files in your folder.\n")


=== 12. Creating Maps and Plots ===

Preparing data for plots...
Clusters and standardized variables calculated.


Map saved as 'Map_Unemployment.png'


`geom_smooth()` using formula = 'y ~ x'


Scatterplot saved as 'Moran_Scatterplot.png'
Done! Check the files in your folder.


In [46]:
# === ДИАГНОСТИКА ИМЕН ===

cat("--- ПРОВЕРКА НЕСОВПАДЕНИЙ ---\n")

# 1. Какие регионы есть в КАРТЕ, но для них нет данных в Excel?
# (Именно они будут белыми на карте)
missing_on_map <- anti_join(russia, data_2019, by = c("name" = "ne_name"))

cat("Эти регионы на карте остались БЕЗ ДАННЫХ (белые):\n")
print(missing_on_map$name)

cat("\n---------------------------\n")

# 2. Какие регионы есть в EXCEL, но они не нашли свою пару на карте?
# (Именно их названия нужно добавить в словарь)
missing_in_data <- anti_join(data_2019, russia, by = c("ne_name" = "name"))

cat("Этим данным из Excel НЕ НАШЛОСЬ места на карте:\n")
print(unique(missing_in_data$ne_name))

--- ПРОВЕРКА НЕСОВПАДЕНИЙ ---
Эти регионы на карте остались БЕЗ ДАННЫХ (белые):
[1] "Yamal-Nenets"  NA              "Khanty-Mansiy"

---------------------------
Этим данным из Excel НЕ НАШЛОСЬ места на карте:
[1] "Central Federal District"         "Northwestern Federal District"   
[3] "Arkhangelsk Oblast"               "Southern Federal District"       
[5] "North Caucasian Federal District" "Volga Federal District"          
[7] "Ural Federal District"            "Siberian Federal District"       
[9] "Far Eastern Federal District"    


In [45]:
# === ПРОВЕРКА: КТО ОСТАЛСЯ БЕЗ ПАРЫ ===

# 1. Какие строки из Excel НЕ нашли себе места на карте?
# (Это "мусор" или Федеральные округа)
unused_data <- anti_join(data_2019, russia, by = c("ne_name" = "name"))

cat("--- СТРОКИ EXCEL, КОТОРЫЕ НЕ ПОПАЛИ В АНАЛИЗ ---\n")
print(unique(unused_data$geoname_name))

# 2. Какие регионы на карте остались БЕЛЫМИ (без данных)?
empty_map_regions <- anti_join(russia, data_2019, by = c("name" = "ne_name"))

cat("\n--- РЕГИОНЫ КАРТЫ, ДЛЯ КОТОРЫХ НЕТ ДАННЫХ ---\n")
print(empty_map_regions$name)

--- СТРОКИ EXCEL, КОТОРЫЕ НЕ ПОПАЛИ В АНАЛИЗ ---
[1] "Central Federal District"         "Northwestern Federal District"   
[3] "Arkhangelsk Oblast"               "Southern Federal District"       
[5] "North Caucasian Federal District" "Volga Federal District"          
[7] "Ural Federal District"            "Siberian Federal District"       
[9] "Far Eastern Federal District"    

--- РЕГИОНЫ КАРТЫ, ДЛЯ КОТОРЫХ НЕТ ДАННЫХ ---
[1] "Yamal-Nenets"  NA              "Khanty-Mansiy"
